In [76]:
import polars as pl
df = pl.read_csv("../data/data.csv")

In [77]:
print(df.columns)
print(df.dtypes)

['', 'timestamp', 'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric', 'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']
[Int64, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64]


# Intervalos anómalos

In [78]:
import datetime 

anomalies = [
    (datetime.datetime(2020, 4, 12, 11, 50), datetime.datetime(2020, 4, 12, 23, 30)),
    (datetime.datetime(2020, 4, 18, 0, 0), datetime.datetime(2020, 4, 18, 23, 59)),
    (datetime.datetime(2020, 4, 19, 0, 0), datetime.datetime(2020, 4, 19, 1, 30)),
    (datetime.datetime(2020, 4, 29, 3, 20), datetime.datetime(2020, 4, 29, 4, 0)),
    (datetime.datetime(2020, 4, 29, 22, 0), datetime.datetime(2020, 4, 29, 22, 20)),
    (datetime.datetime(2020, 5, 13, 14, 0), datetime.datetime(2020, 5, 13, 23, 59)),
    (datetime.datetime(2020, 5, 18, 5, 0), datetime.datetime(2020, 5, 18, 5, 30)),
    (datetime.datetime(2020, 5, 19, 10, 10), datetime.datetime(2020, 5, 19, 11, 0)),
    (datetime.datetime(2020, 5, 19, 22, 10), datetime.datetime(2020, 5, 19, 23, 59)),
    (datetime.datetime(2020, 5, 20, 0, 0), datetime.datetime(2020, 5, 20, 20, 0)),
    (datetime.datetime(2020, 5, 23, 9, 50), datetime.datetime(2020, 5, 23, 10, 10)),
    (datetime.datetime(2020, 5, 29, 23, 30), datetime.datetime(2020, 5, 29, 23, 59)),
    (datetime.datetime(2020, 5, 30, 0, 0), datetime.datetime(2020, 5, 30, 6, 0)),
    (datetime.datetime(2020, 6, 1, 15, 0), datetime.datetime(2020, 6, 1, 15, 40)),
    (datetime.datetime(2020, 6, 3, 10, 0), datetime.datetime(2020, 6, 3, 11, 0)),
    (datetime.datetime(2020, 6, 5, 10, 0), datetime.datetime(2020, 6, 5, 23, 59)),
    (datetime.datetime(2020, 6, 6, 0, 0), datetime.datetime(2020, 6, 6, 23, 59)),
    (datetime.datetime(2020, 6, 7, 0, 0), datetime.datetime(2020, 6, 7, 14, 30)),
    (datetime.datetime(2020, 7, 8, 17, 30), datetime.datetime(2020, 7, 8, 19, 0)),
    (datetime.datetime(2020, 7, 15, 14, 30), datetime.datetime(2020, 7, 15, 19, 0)),
    (datetime.datetime(2020, 7, 17, 4, 30), datetime.datetime(2020, 7, 17, 5, 30)),
]


raros = [
    #NO APARECE COMO ANOMALIA
    (datetime.datetime(2020, 3, 6, 21, 42, 15), datetime.datetime(2020, 3, 6, 23, 14, 0)),
    (datetime.datetime(2020, 3, 11, 5, 15, 10), datetime.datetime(2020, 3, 11, 6, 25, 0)),
    (datetime.datetime(2020, 3, 12, 0, 15, 56), datetime.datetime(2020, 3, 12, 11, 59, 0)),
    (datetime.datetime(2020, 3, 26, 4, 0, 20), datetime.datetime(2020, 3, 26, 5, 20, 0)),
    (datetime.datetime(2020, 3, 27, 7, 12, 0), datetime.datetime(2020, 3, 27, 12, 1, 0)),
    (datetime.datetime(2020, 4, 17, 8, 50, 28), datetime.datetime(2020, 4, 17, 23, 59, 0)),
    (datetime.datetime(2020, 4, 25, 0, 7, 15), datetime.datetime(2020, 4, 25, 1, 10, 0)),
    (datetime.datetime(2020, 5, 19, 1, 35, 28), datetime.datetime(2020, 5, 19, 2, 40, 0)),
    (datetime.datetime(2020, 6, 12, 1, 41, 7), datetime.datetime(2020, 6, 12, 17, 6, 0)),
    (datetime.datetime(2020, 7, 21, 13, 32, 48), datetime.datetime(2020, 7, 21, 22, 3, 0)),
    (datetime.datetime(2020, 7, 22, 6, 40, 46), datetime.datetime(2020, 7, 22, 13, 10, 0)),
    (datetime.datetime(2020, 7, 31, 0, 57, 33), datetime.datetime(2020, 7, 31, 2, 9, 0))
]

anomalies.extend(raros)

# Correción de periodos temporales.
final_anomalies = []
for anomaly in anomalies: 
  new = list(anomaly)
  if anomaly[1].minute == 59:
    new[1] += datetime.timedelta(minutes=1)
  final_anomalies.append(new)

In [79]:
import datetime
df = df.with_columns(pl.col("timestamp").str.to_datetime())
df = df.with_row_index("rownr")

In [80]:
anomalies[1][1] + datetime.timedelta(seconds=59)

datetime.datetime(2020, 4, 18, 23, 59, 59)

# Definimos las anomalias

In [81]:
def is_anomaly(instance_date, anomalies: list[datetime]):
    flag_anomaly = False
    index = 0
    while not flag_anomaly and index < len(anomalies):
        t = anomalies[index]
        if instance_date >= t[0] and instance_date <= t[1] + datetime.timedelta(seconds=59):
            flag_anomaly = True
        index += 1
    return flag_anomaly


df = df.select(pl.all(), pl.lit(0).alias("is_anomaly"))
for anomaly in final_anomalies:
  df = df.with_columns((pl.col("is_anomaly") + df["timestamp"].is_between(anomaly[0], anomaly[1])))
df = df.select(pl.exclude("is_anomaly"), pl.col("is_anomaly") >= 1)
# df = df.select(pl.all(), pl.col("timestamp").map_elements(lambda x: is_anomaly(x, anomalies), return_dtype=pl.Boolean).alias("is_anomaly"))

# Justificamos el cálculo de la ventana

In [82]:
# Create transition states
tdf = df.select(pl.all(), pl.col("is_anomaly").shift(-1).alias("next_is_anomaly"), pl.col("Motor_current").gt(0.05).alias("motor_state"))
tdf = tdf.select(pl.all(), (pl.col("is_anomaly") != pl.col("next_is_anomaly")).alias("transition"))
tdf

rownr,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,is_anomaly,next_is_anomaly,motor_state,transition
u32,i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,bool
0,0,2020-02-01 00:00:00,-0.012,9.358,9.34,-0.024,9.358,53.6,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
1,10,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
2,20,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
3,30,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
4,40,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1516943,15169430,2020-09-01 03:59:10,-0.014,8.918,8.906,-0.022,8.918,59.675,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
1516944,15169440,2020-09-01 03:59:20,-0.014,8.904,8.888,-0.02,8.904,59.6,0.045,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false
1516945,15169450,2020-09-01 03:59:30,-0.014,8.89,8.876,-0.022,8.892,59.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false


In [83]:
# Create groups based on transition
tdf = tdf.select(pl.all(), pl.col("transition").cum_sum().alias("groups"))
tdf["groups"].max()

54

In [84]:
gdf = tdf.select(pl.all(), pl.col("groups").shift(-1).alias("next_group"), pl.col("motor_state").shift(-1).alias("next_motor_state"))
# Filter a ON switch
fdf = gdf.filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1)).with_columns(pl.col("timestamp").shift(-1).alias("next_timestamp"))
# Filter out different time groups
fdf = fdf.filter(pl.col("groups") == pl.col("next_group"))
# Calculate the duration
fdf = fdf.with_columns((pl.col("next_timestamp") - pl.col("timestamp")).alias("duration")).filter(pl.col("duration").is_not_null())
print(fdf["duration"].dt.total_seconds().max(), fdf["duration"].dt.total_seconds().min(), fdf["duration"].dt.total_seconds().mean(), fdf["duration"].dt.total_seconds().mode()[0])

274751 19 1771.898393149235 1249


In [85]:
fdf.filter(pl.col("duration").eq(pl.col("duration").max()))

rownr,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,is_anomaly,next_is_anomaly,motor_state,transition,groups,next_group,next_motor_state,next_timestamp,duration
u32,i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,bool,u32,u32,bool,datetime[μs],duration[μs]
890739,8907390,2020-06-05 09:48:20,-0.014,8.154,8.142,-0.018,8.156,60.575,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,false,false,false,38,38,true,2020-06-08 14:07:31,3d 4h 19m 11s


In [86]:
import plotly.express as px
data_anomaly = df.filter(pl.col("timestamp").is_between(
datetime.datetime(2020,6,5,00,00,00), datetime.datetime(2020,6,8,15,00,00)
))
px.line(data_anomaly, x="timestamp", y=["TP3", "Motor_current"])

In [87]:
data_anomaly.select(pl.col("is_anomaly"), pl.col("is_anomaly").shift(-1).alias("next_is_anomaly"), pl.col("timestamp"), pl.col("timestamp").shift(-1).alias("next_timestamp")).filter(~pl.col("is_anomaly").eq(pl.col("next_is_anomaly")))

is_anomaly,next_is_anomaly,timestamp,next_timestamp
bool,bool,datetime[μs],datetime[μs]
false,true,2020-06-05 09:59:54,2020-06-05 10:00:04
true,false,2020-06-07 14:19:39,2020-06-08 11:48:04


## Para cada grupo

In [88]:
gdf = tdf.select(pl.all(), pl.col("groups").shift(-1).alias("next_group"), pl.col("motor_state").shift(-1).alias("next_motor_state"))
# Filter a ON switch
durations = []
for label, group in gdf.filter(pl.col("groups") == pl.col("next_group")).group_by("groups"):
  tmp = group.filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1)).with_columns(pl.col("timestamp").shift(-1).alias("next_timestamp"))
  tmp = tmp.with_columns((pl.col("next_timestamp") - pl.col("timestamp")).alias("duration")).filter(pl.col("duration").is_not_null())
  durations += tmp["duration"].to_list()
durations = [x.total_seconds() for x in durations]
print(max(durations), min(durations), sum(durations) / len(durations), durations[len(durations)//2])

224082.0 19.0 1695.8500048323185 1170.0


## Para el primer tramo no-anómalo

In [89]:
subdf = df.filter(pl.col("timestamp").lt(anomalies[0][0]))
subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))["timestamp"].diff().dt.total_seconds().mode()[0]

1586

## Para cada grupo evitando saltos temporales

In [90]:
durations = []
subdf = df.filter(pl.col("timestamp").lt(final_anomalies[0][0]))
subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))
subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
print(len(durations), len(subdf))
durations += subdf["duration"].to_list()
for anomaly, next_anomaly, next_next_anomaly in zip(final_anomalies, final_anomalies[1:], final_anomalies[2:]):
  subdf = df.filter(
    (pl.col("timestamp").lt(next_anomaly[0]) & (pl.col("timestamp").gt(anomaly[1])))
  )
  subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
  subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))
  subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
  print(len(durations), len(subdf))
  durations += subdf["duration"].to_list()
  subdf = df.filter(
    (pl.col("timestamp").gt(next_anomaly[1]) & (pl.col("timestamp").lt(next_next_anomaly[0])))
  )
  subdf = subdf.with_columns(pl.col("Motor_current").lt(0.05).alias("motor_state"))
  subdf = subdf.with_columns(pl.col("motor_state").shift(-1).alias("next_motor_state")).filter((pl.col("motor_state") == 0) & (pl.col("next_motor_state") == 1))
  subdf = subdf.with_columns(pl.col("timestamp").diff().dt.total_seconds().alias("duration")).filter(pl.col("duration").is_not_null())
  print(len(durations), len(subdf))
  durations += subdf["duration"].to_list()
print(max(durations), min(durations), sum(durations) / len(durations), sorted(durations)[len(durations)//2])

0 3165
3165 147
3312 0
3312 0
3312 311
3623 311
3934 38
3972 38
4010 515
4525 515
5040 182
5222 182
5404 100
5504 100
5604 43
5647 43
5690 0
5690 0
5690 193
5883 193
6076 269
6345 269
6614 0
6614 0
6614 145
6759 145
6904 83
6987 83
7070 115
7185 115
7300 0
7300 0
7300 0
7300 0
7300 1745
9045 1745
10790 379
11169 379
11548 97
11645 97
11742 0
11742 0
11742 230
11972 230
12202 22
12224 22
12246 612
12858 612
13470 53
13523 53
13576 802
14378 802
15180 230
15410 230
15640 902
16542 902
17444 1106
18550 1106
19656 2316
21972 2316
24288 24
24312 24
24336 518
274930 20 1781.5975698076768 1289


In [91]:
df = df.with_columns(pl.col("timestamp").diff().cum_sum().alias("diff"))

In [92]:
df = df.with_columns((pl.col("diff").dt.total_seconds() // (1249 * 2)).alias("id"))
df = df.with_columns(pl.coalesce(pl.col("id").cast(pl.UInt64), 0).alias("id"))

In [93]:
df.group_by("id").len()

id,len
u64,u32
3835,252
6059,253
4338,252
676,252
1349,252
…,…
3239,252
5621,252
6541,252


In [94]:
start = 50000
end = start + 10000
px.line(df[start : end], x="timestamp", y="TP3", color="id")

In [95]:
with pl.Config(tbl_rows=1000):
  print(df.filter(pl.col("timestamp").gt(datetime.datetime(2020,2, 1, 19, 40, 00)) & pl.col("timestamp").lt(datetime.datetime(2020, 2, 2, 6, 00, 00))))

shape: (549, 21)
┌───────┬───────┬─────────────────┬────────┬───┬────────────────┬────────────┬───────────────┬─────┐
│ rownr ┆       ┆ timestamp       ┆ TP2    ┆ … ┆ Caudal_impulse ┆ is_anomaly ┆ diff          ┆ id  │
│ ---   ┆ ---   ┆ ---             ┆ ---    ┆   ┆ s              ┆ ---        ┆ ---           ┆ --- │
│ u32   ┆ i64   ┆ datetime[μs]    ┆ f64    ┆   ┆ ---            ┆ bool       ┆ duration[μs]  ┆ u64 │
│       ┆       ┆                 ┆        ┆   ┆ f64            ┆            ┆               ┆     │
╞═══════╪═══════╪═════════════════╪════════╪═══╪════════════════╪════════════╪═══════════════╪═════╡
│ 7113  ┆ 71130 ┆ 2020-02-01      ┆ -0.012 ┆ … ┆ 1.0            ┆ false      ┆ 19h 40m 4s    ┆ 28  │
│       ┆       ┆ 19:40:04        ┆        ┆   ┆                ┆            ┆               ┆     │
│ 7114  ┆ 71140 ┆ 2020-02-01      ┆ -0.018 ┆ … ┆ 1.0            ┆ false      ┆ 23h 15m 33s   ┆ 33  │
│       ┆       ┆ 23:15:33        ┆        ┆   ┆                ┆         

In [96]:
df.filter(pl.col("timestamp").diff().dt.total_seconds().gt(10))

rownr,,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,is_anomaly,diff,id
u32,i64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,duration[μs],u64
4654,46540,2020-02-01 12:53:47,-0.012,9.124,9.108,-0.012,9.13,59.675,0.0375,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,12h 53m 47s,18
7114,71140,2020-02-01 23:15:33,-0.018,9.99,9.98,-0.022,9.992,44.85,3.97,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,23h 15m 33s,33
7144,71440,2020-02-02 04:34:27,9.952,9.63,-0.012,-0.024,9.63,31.975,6.275,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,false,1d 4h 34m 27s,41
8254,82540,2020-02-02 07:41:52,-0.012,8.31,8.294,-0.022,8.312,53.1,3.8125,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,1d 7h 41m 52s,45
31804,318040,2020-02-05 00:43:00,10.412,10.032,-0.012,-0.022,10.03,56.6,6.3325,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,false,4d 43m,139
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1502468,15024680,2020-08-30 04:26:10,9.264,8.844,-0.01,-0.022,8.842,51.775,6.0075,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,false,211d 4h 26m 10s,7304
1509758,15097580,2020-08-31 01:49:32,-0.012,9.878,9.866,-0.02,9.878,64.45,3.7775,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,212d 1h 49m 32s,7335
1511498,15114980,2020-08-31 06:42:31,-0.012,9.596,9.58,-0.022,9.596,65.775,3.6775,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,false,212d 6h 42m 31s,7342


# Verificar valores raros

In [97]:
px.line(df.filter(pl.col("timestamp").is_between(*raros[5])), x="timestamp", y="TP3", color="id")

# Revisar datos procesados

In [98]:
import polars as pl 

df = pl.read_csv("../data/data.csv")
df

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""2020-02-01 00:00:00""",-0.012,9.358,9.34,-0.024,9.358,53.6,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
10,"""2020-02-01 00:00:10""",-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
20,"""2020-02-01 00:00:19""",-0.012,9.338,9.322,-0.022,9.338,53.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
30,"""2020-02-01 00:00:29""",-0.012,9.328,9.312,-0.022,9.328,53.425,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
40,"""2020-02-01 00:00:39""",-0.012,9.318,9.302,-0.022,9.318,53.475,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15169430,"""2020-09-01 03:59:10""",-0.014,8.918,8.906,-0.022,8.918,59.675,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
15169440,"""2020-09-01 03:59:20""",-0.014,8.904,8.888,-0.02,8.904,59.6,0.045,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
15169450,"""2020-09-01 03:59:30""",-0.014,8.89,8.876,-0.022,8.892,59.6,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [99]:
df.filter(pl.col("timestamp").str.to_datetime().diff().dt.total_seconds() > 1260 * 2)

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
71140,"""2020-02-01 23:15:33""",-0.018,9.99,9.98,-0.022,9.992,44.85,3.97,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
71440,"""2020-02-02 04:34:27""",9.952,9.63,-0.012,-0.024,9.63,31.975,6.275,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
319540,"""2020-02-05 04:28:42""",-0.024,10.122,10.122,-0.022,10.122,42.25,3.8725,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
398140,"""2020-02-06 07:52:21""",7.768,7.436,-0.01,-0.022,7.442,35.275,5.6925,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
438940,"""2020-02-06 23:33:19""",10.174,9.896,-0.008,-0.02,9.896,43.35,6.265,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
14953580,"""2020-08-29 04:55:08""",8.838,8.386,-0.01,-0.024,8.384,44.65,5.9425,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
15024680,"""2020-08-30 04:26:10""",9.264,8.844,-0.01,-0.022,8.842,51.775,6.0075,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
15097580,"""2020-08-31 01:49:32""",-0.012,9.878,9.866,-0.02,9.878,64.45,3.7775,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


# Entrenar modelo Naive Bayes

In [150]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
import numpy as np

np.random.seed(140421)

# Load the training and testing data
train_data = pd.read_csv('../data/train.csv')
train_data.reset_index()
test_data = pd.read_csv('../data/test.csv')
test_data.reset_index()

target_column = 'is_anomaly'
fold_column = 'fold'
feature_columns = [col for col in train_data.columns if col not in [target_column, fold_column]]

X_train = train_data[feature_columns]
y_train = train_data[target_column]

def get_cv_iterable(
  folds: list,
  fold_column: str,
  train: pd.DataFrame,
): 
  for fold in folds:
    test_indexes = train[train[fold_column] == fold].index
    train_indexes = train[train[fold_column] != fold].index
    yield (train_indexes, test_indexes)

X_test = test_data[feature_columns]
y_test = test_data[target_column]

# Set up cross-validation using the 'folds' column
folds = train_data[fold_column].unique()
cv = GridSearchCV(
  estimator = GaussianNB(), 
  param_grid={
    "var_smoothing": np.linspace(1e-12, 1e-1, 250)
  },
  cv = get_cv_iterable(folds, fold_column, train_data),
  scoring=[
    "f1",
    "recall",
    "precision", 
    "accuracy"
  ],
  refit="f1",
)

cv.fit(X_train, y_train)
with pd.option_context("display.max_columns", 33):
  display(pd.DataFrame(cv.cv_results_))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_recall,split1_test_recall,split2_test_recall,...,std_test_recall,rank_test_recall,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
0,0.010367,0.001049,0.016863,0.000994,1.000000e-12,{'var_smoothing': 1e-12},0.549020,0.757062,0.476596,0.610526,0.598301,0.103193,202,0.933333,0.930556,0.933333,...,0.014878,1,0.388889,0.638095,0.320000,0.446154,0.448284,0.118340,250,0.946356,0.949292,0.944843,0.939344,0.944959,0.003615,250
1,0.011170,0.001441,0.019964,0.003799,4.016064e-04,{'var_smoothing': 0.00040160642669879516},0.555556,0.760234,0.493392,0.630435,0.609904,0.099435,165,0.916667,0.902778,0.933333,...,0.023845,2,0.398551,0.656566,0.335329,0.467742,0.464547,0.120348,248,0.948688,0.951651,0.948430,0.944262,0.948258,0.002631,248
2,0.010128,0.001701,0.018421,0.001594,8.032129e-04,{'var_smoothing': 0.0008032128523975903},0.548223,0.760234,0.495575,0.630435,0.608617,0.099862,167,0.900000,0.902778,0.933333,...,0.027030,3,0.394161,0.656566,0.337349,0.467742,0.463954,0.120430,249,0.948105,0.951651,0.948879,0.944262,0.948224,0.002640,249
3,0.010385,0.001020,0.018435,0.001876,1.204819e-03,{'var_smoothing': 0.0012048192780963855},0.551020,0.760234,0.497778,0.630435,0.609867,0.098818,166,0.900000,0.902778,0.933333,...,0.027030,3,0.397059,0.656566,0.339394,0.467742,0.465190,0.119476,247,0.948688,0.951651,0.949327,0.944262,0.948482,0.002674,247
4,0.010202,0.001563,0.017506,0.000344,1.606426e-03,{'var_smoothing': 0.0016064257037951806},0.551020,0.764706,0.497778,0.630435,0.610985,0.100524,163,0.900000,0.902778,0.933333,...,0.027030,3,0.397059,0.663265,0.339394,0.467742,0.466865,0.122163,246,0.948688,0.952830,0.949327,0.944262,0.948777,0.003046,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.009649,0.000889,0.016030,0.000971,9.839357e-02,{'var_smoothing': 0.09839357429720481},0.516556,0.713287,0.513089,0.647887,0.597705,0.086056,203,0.650000,0.708333,0.816667,...,0.062465,245,0.428571,0.718310,0.374046,0.560976,0.520476,0.132916,9,0.957434,0.951651,0.958296,0.959016,0.956599,0.002911,9
246,0.009917,0.001933,0.019305,0.002360,9.879518e-02,{'var_smoothing': 0.09879518072290361},0.516556,0.713287,0.513089,0.647887,0.597705,0.086056,203,0.650000,0.708333,0.816667,...,0.062465,245,0.428571,0.718310,0.374046,0.560976,0.520476,0.132916,9,0.957434,0.951651,0.958296,0.959016,0.956599,0.002911,9
247,0.009272,0.002070,0.015450,0.000613,9.919679e-02,{'var_smoothing': 0.09919678714860242},0.516556,0.713287,0.513089,0.647887,0.597705,0.086056,203,0.650000,0.708333,0.816667,...,0.062465,245,0.428571,0.718310,0.374046,0.560976,0.520476,0.132916,9,0.957434,0.951651,0.958296,0.959016,0.956599,0.002911,9
248,0.009478,0.001425,0.017376,0.002218,9.959839e-02,{'var_smoothing': 0.09959839357430121},0.516556,0.713287,0.513089,0.647887,0.597705,0.086056,203,0.650000,0.708333,0.816667,...,0.062465,245,0.428571,0.718310,0.374046,0.560976,0.520476,0.132916,9,0.957434,0.951651,0.958296,0.959016,0.956599,0.002911,9


In [149]:
print(cv.best_estimator_, cv.best_score_)

GaussianNB(var_smoothing=np.float64(0.037737737800000004)) 0.6222177714511882
